## Import Libraries

In [1]:
from os import path, makedirs
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio

from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt
from brainpipe.visual import *
from brainpipe.statistics import *
from scipy.stats import *

## Power Decoding - Partial//Detailed Encoding
### For ALL time points

In [5]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.model_selection import permutation_test_score

st = study('Olfacto')
exp = 'Enc' #'Enc'
conds, subjects = ['low','high'],['CHAF','VACJ','SEMC','LEFC','PIRJ','FERJ']
freqs = ['0_theta','1_alpha','2_beta','3_gamma']
path_pow = path.join(st.path, 'feature/TPSim_'+exp+'_By_Odor_By_Cond/TPS_by_cond/')
save_path = path.join(st.path, 'classified/Classif_TPSim_'+exp[0]+'_by_cond/')
nperm = 1000

for su in subjects:
    for freq in freqs:
        pow_list = []
        #=========================== Load Power files (nfreq, nelec, nwin, ntrial) =================================    
        mat0 = np.load(path.join(path_pow, 'TPS_spear_'+su+'_cond_'+conds[0]+'_'+freq[2:]+'.npz'),
                       allow_pickle=True)
        pow_list.append(mat0['tps'])
        nelecs = mat0['tps'].shape[0]
        mat1 = np.load(path.join(path_pow, 'TPS_spear_'+su+'_cond_'+conds[1]+'_'+freq[2:]+'.npz'),
                       allow_pickle=True)
        pow_list.append(mat1['tps'])
        print (su,mat0.files, 'TPS shape: ', [pow.shape for pow in pow_list])
        
        #=========================== Create dict for all results =================================    
        name_classif = (save_path+'/'+su +'_LDA_classif_'+freq+'_'+conds[0]+'_'+conds[1]+'.npz')
        if not path.exists(name_classif):
            kwargs = {}
            kwargs['names'], kwargs['channels'] = mat0['label'], mat0['channel']
            kwargs['xyz'] = mat0['xyz']

            # =========================== Select Power for 1 elec 1 freq =================================                 
            permut,auc = np.zeros((nelecs,nperm)),np.zeros((nelecs))
            for elec_num in range(nelecs):
                print('--» processing',su, 'elec', elec_num,'/',nelecs, 'freq',freq)
                pow_data_elec = [power[elec_num][:,np.newaxis] for power in pow_list]

                # create a data matrix, concatenate along the trial dimension
                x = np.concatenate(pow_data_elec, axis=0)
                print ('Size of the concatenated data: ', x.shape)
                y = np.hstack([np.array([i]*len(power)) for i, power in enumerate(pow_data_elec)])
                print ('Size of label for classif: ', len(y))

                clf = LDA()
                cv = SKFold(5)
                score, permutation_scores, pvalue = permutation_test_score(
                clf, x, y, scoring="roc_auc", cv=cv, n_permutations=nperm, n_jobs=-1)
                permut[elec_num] += permutation_scores
                auc[elec_num] += score
                print("Classification score %s (pvalue : %s)" % (score, pvalue))

            kwargs['perm'], kwargs['auc'] = permut, auc
            #Save plots
            np.savez(name_classif, **kwargs)
            del x, y, pow_data_elec, permutation_scores, score
        else:
            print(name_classif, 'already computed')
    del pow_list

-> Olfacto loaded
CHAF ['tps', 'pval', 'label', 'channel', 'xyz'] TPS shape:  [(57, 66), (57, 66)]
--» processing CHAF elec 0 / 57 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.5824779615988407 (pvalue : 0.11988011988011989)
--» processing CHAF elec 1 / 57 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.5772551624199975 (pvalue : 0.14685314685314685)
--» processing CHAF elec 2 / 57 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.35009358773095034 (pvalue : 0.988011988011988)
--» processing CHAF elec 3 / 57 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.3842229199372057 (pvalue : 0.949050949050949)
--» processing CHAF elec 4 / 57 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.6883951

Classification score 0.29269411906774545 (pvalue : 0.999000999000999)
--» processing CHAF elec 44 / 57 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.5312643400555488 (pvalue : 0.46053946053946054)
--» processing CHAF elec 45 / 57 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.733643279797126 (pvalue : 0.000999000999000999)
--» processing CHAF elec 46 / 57 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.4146298756188866 (pvalue : 0.8611388611388612)
--» processing CHAF elec 47 / 57 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.5525661152034779 (pvalue : 0.3096903096903097)
--» processing CHAF elec 48 / 57 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.5400313971742543 (pvalue : 0.3

Classification score 0.3311496196111581 (pvalue : 1.0)
--» processing CHAF elec 30 / 57 freq 1_alpha
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.6883830455259027 (pvalue : 0.000999000999000999)
--» processing CHAF elec 31 / 57 freq 1_alpha
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.24045405144306248 (pvalue : 1.0)
--» processing CHAF elec 32 / 57 freq 1_alpha
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.43225153966912205 (pvalue : 0.7922077922077922)
--» processing CHAF elec 33 / 57 freq 1_alpha
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.46114599686028257 (pvalue : 0.6633366633366633)
--» processing CHAF elec 34 / 57 freq 1_alpha
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.5518355271102524 (pvalue : 0.30569430569430567)
--» proce

Classification score 0.6228535201062674 (pvalue : 0.016983016983016984)
--» processing CHAF elec 17 / 57 freq 2_beta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.45641830696775754 (pvalue : 0.6103896103896104)
--» processing CHAF elec 18 / 57 freq 2_beta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.4130720927424224 (pvalue : 0.8771228771228772)
--» processing CHAF elec 19 / 57 freq 2_beta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.6885279555609225 (pvalue : 0.000999000999000999)
--» processing CHAF elec 20 / 57 freq 2_beta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.679477116290303 (pvalue : 0.000999000999000999)
--» processing CHAF elec 21 / 57 freq 2_beta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.3959727086100712 (pvalue : 0.90

Classification score 0.6205953387272067 (pvalue : 0.027972027972027972)
--» processing CHAF elec 3 / 57 freq 3_gamma
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.3936088636638087 (pvalue : 0.8291708291708292)
--» processing CHAF elec 4 / 57 freq 3_gamma
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.30266272189349114 (pvalue : 0.998001998001998)
--» processing CHAF elec 5 / 57 freq 3_gamma
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.4044257939862336 (pvalue : 0.7672327672327672)
--» processing CHAF elec 6 / 57 freq 3_gamma
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.6422775027170632 (pvalue : 0.004995004995004995)
--» processing CHAF elec 7 / 57 freq 3_gamma
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.6023547880690737 (pvalue : 0.0499

...

Classification score 0.6082417582417582 (pvalue : 0.013986013986013986)
--» processing PIRJ elec 9 / 30 freq 3_gamma
Size of the concatenated data:  (186, 1)
Size of label for classif:  186
Classification score 0.4184981684981685 (pvalue : 0.8961038961038961)
--» processing PIRJ elec 10 / 30 freq 3_gamma
Size of the concatenated data:  (186, 1)
Size of label for classif:  186
Classification score 0.5801282051282051 (pvalue : 0.0939060939060939)
--» processing PIRJ elec 11 / 30 freq 3_gamma
Size of the concatenated data:  (186, 1)
Size of label for classif:  186
Classification score 0.649496336996337 (pvalue : 0.002997002997002997)
--» processing PIRJ elec 12 / 30 freq 3_gamma
Size of the concatenated data:  (186, 1)
Size of label for classif:  186
Classification score 0.5527472527472528 (pvalue : 0.25774225774225773)
--» processing PIRJ elec 13 / 30 freq 3_gamma
Size of the concatenated data:  (186, 1)
Size of label for classif:  186
Classification score 0.5531593406593407 (pvalue : 0.

Classification score 0.5477055911121844 (pvalue : 0.3196803196803197)
--» processing FERJ elec 22 / 45 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.354588817775631 (pvalue : 0.987012987012987)
--» processing FERJ elec 23 / 45 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.6214527231010747 (pvalue : 0.012987012987012988)
--» processing FERJ elec 24 / 45 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.5986776959304432 (pvalue : 0.059940059940059943)
--» processing FERJ elec 25 / 45 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.60810892404299 (pvalue : 0.03296703296703297)
--» processing FERJ elec 26 / 45 freq 0_theta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.47226784204806177 (pvalue : 0.5

Classification score 0.3561496196111581 (pvalue : 0.9600399600399601)
--» processing FERJ elec 20 / 45 freq 1_alpha
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.6255102040816327 (pvalue : 0.015984015984015984)
--» processing FERJ elec 21 / 45 freq 1_alpha
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.7345368916797488 (pvalue : 0.000999000999000999)
--» processing FERJ elec 22 / 45 freq 1_alpha
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.4661997343316024 (pvalue : 0.6073926073926074)
--» processing FERJ elec 23 / 45 freq 1_alpha
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.661592802801594 (pvalue : 0.002997002997002997)
--» processing FERJ elec 24 / 45 freq 1_alpha
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.444698707885521 (pvalue : 0

Classification score 0.6259751237773216 (pvalue : 0.013986013986013986)
--» processing FERJ elec 18 / 45 freq 2_beta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.4196051201545707 (pvalue : 0.7532467532467533)
--» processing FERJ elec 19 / 45 freq 2_beta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.5764158918005071 (pvalue : 0.14485514485514486)
--» processing FERJ elec 20 / 45 freq 2_beta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.39866562009419154 (pvalue : 0.9010989010989011)
--» processing FERJ elec 21 / 45 freq 2_beta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.5092621664050235 (pvalue : 0.5164835164835165)
--» processing FERJ elec 22 / 45 freq 2_beta
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.6256128486897718 (pvalue : 0.0209

Classification score 0.6039216278227266 (pvalue : 0.04495504495504495)
--» processing FERJ elec 16 / 45 freq 3_gamma
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.35410578432556455 (pvalue : 0.993006993006993)
--» processing FERJ elec 17 / 45 freq 3_gamma
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.536734693877551 (pvalue : 0.43156843156843155)
--» processing FERJ elec 18 / 45 freq 3_gamma
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.6267600531336796 (pvalue : 0.01998001998001998)
--» processing FERJ elec 19 / 45 freq 3_gamma
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.6985810892404298 (pvalue : 0.000999000999000999)
--» processing FERJ elec 20 / 45 freq 3_gamma
Size of the concatenated data:  (132, 1)
Size of label for classif:  132
Classification score 0.6592531095278348 (pvalue : 0